In [5]:
# %edit
# %load_ext autoreload
# %autoreload 2

IPython will make a temporary file named: C:\Users\yhu42\AppData\Local\Temp\ipython_edit_vfhz_ifa\ipython_edit_rzpz52fn.py


In [7]:
#!pip install utils

In [1]:
# GPT-3 code was run in April, 2023
import openai
# import jax.numpy as jnp
# from jax.scipy.special import logsumexp
from typing import Optional, Tuple, Dict
import numpy as np
import pandas as pd
import time
# import hydra
# from cdt.causality.pairwise import RECI
import os
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
import re
from diskcache import Cache
from tqdm import tqdm
import json
import time
from transformers import pipeline, set_seed
import random

from utils import *

In [2]:
cache = Cache("./")


tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")


In [3]:
@cache.memoize()
def get_answer_logits(prompt, n_tokens = 16, temp=1.0, logprobs = 5, engine = "text-davinci-002"):
    """Gets logits for the possible next tokens
    Args:
        prompt: prompt for GPT-3
    """
    response = openai.Completion.create(
            engine=engine,
            prompt=prompt,
            max_tokens =  n_tokens,
            temperature=temp,
            top_p=1,
            n=1,
            logprobs=logprobs,  # 100 is the maximum here,
            presence_penalty=0,
            frequency_penalty=0,
            best_of=1,
            echo=True,
        )

    text_generate = response['choices'][0]['text'].replace(prompt, "")
    logprobs_generate = response['choices'][0]['logprobs']
    # tokenized prompt
    prompt_tokens = tokenizer.encode(prompt)
    # tokenized answer
    text_generate_tokes = logprobs_generate['tokens'][len(prompt_tokens):]
    # logprob of each answer token
    text_generate_logprobs = logprobs_generate['top_logprobs'][len(prompt_tokens):]

    return {
        'answer': text_generate,
        'answer_tokens': text_generate_tokes,
        'answer_logprobs': text_generate_logprobs
    }

@cache.memoize()
def get_place_recognition_answer_logits(row, recog_prompt, tokenizer, 
                                    text_key = "text",
                                    engine = "text-davinci-002", 
                                    n_tokens = 16, 
                                    temp=1.0, 
                                    logprobs = 5):
    
    text = row[text_key]
    # phrase = row["phrase"]
    # dbpediaID = row["dbpediaID"]
    # ID = row["ID"]
    
    text_ = text.capitalize()

    recog_prompt_ = recog_prompt.replace("{TEXT}", text_)
    

    # n_answer_tokens = len(tokenizer.encode(phrase)) + 1

    res_dict = get_answer_logits(prompt = recog_prompt_, 
                          n_tokens = n_tokens, 
                          temp=temp, 
                          logprobs = logprobs, 
                          engine = engine)
    res_dict = {**res_dict,
                **dict(row)
               }
    return res_dict

In [4]:

# 22 examples
recog_prompt = """This is a set of location description recognition problems.
The `Sentence` is a sentence containing location descriptions.
The goal is to infer which parts of the sentence represent location descriptions and the categories of the location descriptions. Split different location descriptions with `;`.
--

--
Sentence: Papa stranded in home. Water rising above waist. HELP 812 Wood Ln, 77828 #houstonflood
Q: Which parts of this sentence represent location descriptions?
A: C1: 812 Wood Ln, 77828 
--

--
Sentence: Anyone doing high water rescues in the Pasadena/Deer Park area? My daughter has been stranded in a parking lot all night
Q: Which parts of this sentence represent location descriptions?
A: C10: Pasadena/Deer Park
--

--
Sentence: Allen Parkway, Memorial, Waugh overpass, Spotts park and Buffalo Bayou park completely under water
Q: Which parts of this sentence represent location descriptions?
A: C2: Allen Parkway; C2: Memorial; C2: Waugh overpass; C7: Spotts park; C7: Buffalo Bayou park
--

--
Sentence: Streets Flooded: Almeda Genoa Rd. from Windmill Lakes Blvd. to Rowlett Rd. HurricaneHarvey Houston
Q: Which parts of this sentence represent location descriptions?
A: C11: Almeda Genoa Rd. from Windmill Lakes Blvd. to Rowlett Rd.; C9: Houston
--

--
Sentence: Cleaning supply drive is underway. 9-11 am today at Preston Hollow Presbyterian Church
Q: Which parts of this sentence represent location descriptions?
A: C8: Preston Hollow Presbyterian Church
--

--
Sentence: #Harvey LIVE from San Antonio, TX. Fatal car accident at Ingram Rd., Strong winds.
Q: Which parts of this sentence represent location descriptions?
A: C9: San Antonio; C9: TX; C2: Ingram Rd.
--

--
Sentence: 9:00AM update video from Hogan St over White Oak Bayou, I-10, I-45: water down about 4’ since last night.
Q: Which parts of this sentence represent location descriptions?
A: C5: Hogan St over White Oak Bayou; C3: I-10; C3: I-45
--

--
Sentence: Left Corpus bout to be in San Angelo #HurricaneHarvey Y’all be safe Avoided highway 37 Took the back road 
Q: Which parts of this sentence represent location descriptions?
A: C9: Corpus; C9: San Angelo; C3: highway 37
--

--
Sentence: Need trailers/trucks to move dogs from Park Location: Whites Park Pavillion off I-10 exit 61 Anahuac TX
Q: Which parts of this sentence represent location descriptions?
A: C7: Whites Park Pavillion; C3: I-10; C4: exit 61; C9: Anahuac; C9: TX
--

--
Sentence: Townsend exit, Sorters road and Hamblen road is flooded coming from 59 southbound HurricaneHarvery Harvey2017 
Q: Which parts of this sentence represent location descriptions?
A: C4: Townsend exit; C5: Sorters road and Hamblen road; C3: 59 southbound
--

--
Sentence: #Harvey does anyone know about the flooding conditions around Cypress Ridge High School?! #HurricaneHarvey
Q: Which parts of this sentence represent location descriptions?
A: C8: Cypress Ridge High School
--

--
Sentence: FYI to any of you in NW Houston/Lakewood Forest, Projections are showing Cypress Creek overflowing at Grant Rd
Q: Which parts of this sentence represent location descriptions?
A: C10: NW Houston/Lakewood Forest; C5: Cypress Creek overflowing at Grant Rd
--

--
Sentence: #HurricaneHarvey family needs rescuing at 11800 Grant Rd. Apt. 1009 in Cypress, Texas 77429, 2 elderly, one is 90 not steady in her feet
Q: Which parts of this sentence represent location descriptions?
A: C1: 11800 Grant Rd. Apt. 1009 in Cypress, Texas 77429
--

--
Sentence: Guys, this is I-45 @ Main Street in Houston. Crazy. #hurricane #harvey. . .
Q: Which parts of this sentence represent location descriptions?
A: C5: I-45 @ Main Street; C9: Houston
--

--
Sentence: Frontage Rd at the river #hurricaneHarvey #hurricaneharvey @ San Jacinto River
Q: Which parts of this sentence represent location descriptions?
A: C2: Frontage Rd; C6: San Jacinto River
--

--
Sentence: Pictures of downed trees and damaged apartment building on Airline Road in Corpus Christi.
Q: Which parts of this sentence represent location descriptions?
A: C2: Airline Road; C9: Corpus Christi
--

--
Sentence: Buffalo Bayou holding steady at 10,000 cfs at the gage near Terry Hershey Park
Q: Which parts of this sentence represent location descriptions?
A: C6: Buffalo Bayou; C7: Terry Hershey Park
--

--
Sentence: Major flooding at Clay Rd & Queenston in west Houston. Lots of rescues going on for ppl trapped...
Q: Which parts of this sentence represent location descriptions?
A: C5: Clay Rd & Queenston; C9: Houston
--

--
Sentence: HELP! A pregnant lady is stuck in her car on I-45 between Cypress Hill & Huffmeister exits #harvey
Q: Which parts of this sentence represent location descriptions?
A: C11: I-45 between Cypress Hill & Huffmeister exits
--

--
Sentence: If you need a place to escape #HurricaneHarvey, The Willie De Leon Civic Center: 300 E. Main St in Uvalde is open as a shelter
Q: Which parts of this sentence represent location descriptions?
A: C7: The Willie De Leon Civic Center; C1: 300 E. Main St in Uvalde
--

--
Sentence: Houston’s Buffalo Bayou Park - always among the first to flood. #Harvey
Q: Which parts of this sentence represent location descriptions?
A: C9: Houston; C7: Buffalo Bayou Park
--

--
Sentence: #HurricaneHarvey INTENSE eye wall of category 4 Hurricane Harvey from Rockport, TX
Q: Which parts of this sentence represent location descriptions?
A: C9: Rockport; C9: TX
--

--
Sentence: {TEXT}
Q: Which parts of this sentence represent location descriptions?
A:"""

In [5]:
len(tokenizer.encode(recog_prompt))

1510

In [4]:
def load_HarveyTweet2017_data(anno_file, data_dir, sep = "||"):
    with open(anno_file, 'r') as f:
        i = 0
        data_list = []
        for line in f:
            line = line.split("\n")[0]
            parts = line.split(sep)
            # print(parts)
            if i == 0:
                cols = parts
            else:
                file = os.path.join(data_dir, parts[0])
                text = parts[1]
                anno = json.loads(parts[2])
                item = {
                    "ID": i - 1,
                    cols[0]: file,
                    cols[1]: text,
                    cols[2]: anno,
                }

                data_list.append(item)

            i += 1
    return data_list

def read_res_file(res_file):
    res_dict_list = []
    
    if os.path.exists(res_file):
        with open(res_file, 'r') as f:
            for line in f:
                res_dict = json.loads(line)
                res_dict_list.append(res_dict)
    return res_dict_list

In [7]:
engine = "text-davinci-002"  

set_seed(42)

data_dir = "data/"
anno_file = os.path.join(data_dir, 'tweetTextAndAnnotation.txt')
data_list = load_HarveyTweet2017_data(anno_file, data_dir, sep = "||")



n_tokens = 30
temp=0.2
logprobs = 5
logprobs_topk = 20

res_dir = "result/"  #random22_outside_1000
if not os.path.exists(res_dir):
    os.mkdir(res_dir)
res_file = "{res_dir}/placerecognize_{engine}_{temp:.3f}_{n_tokens:d}_{logprobs:d}_22examples.json".format(
        res_dir = res_dir,
        engine = engine,
        temp = temp,
        n_tokens = n_tokens,
        logprobs = logprobs
    )
res_dict_list = read_res_file(res_file)
process_ids = [res_dict['ID'] for res_dict in res_dict_list]

cur_data_list = [item for item in data_list if item["ID"] not in process_ids]

In [8]:
cur_data_list[29]

{'ID': 29,
 'tweetFileID': 'data/124.txt',
 'tweetFulltext': 'houstonflood ANYONE gets info on extent of buffalo bayou flooding near Allen Pkwy and Studemont St & Memorial Ct',
 'tweetAnnotation': [{'locationDesc': 'buffalo bayou',
   'locationCate': 'C6',
   'startIndex': 43,
   'endIndex': 56},
  {'locationDesc': 'Allen Pkwy and Studemont St & Memorial Ct',
   'locationCate': 'C11',
   'startIndex': 71,
   'endIndex': 112}]}

In [5]:
openai.api_key = ""   ## note: An OpenAI API key needs to be added here

In [ ]:
res_dict = get_place_recognition_answer_logits(cur_data_list[0], recog_prompt, tokenizer, 
                                                   text_key = "tweetFulltext",
                                                    engine = engine, 
                                                    n_tokens = n_tokens, 
                                                    temp=temp, 
                                                    logprobs = logprobs)

res_dict

In [16]:
res_dict['answer']

' CX: Houston; CX: i-45 & n. main street'

In [17]:
len(cur_data_list)

1000

In [18]:
#count = 0
for i, row in (pbar := tqdm(enumerate(cur_data_list), total= len(cur_data_list) )):
    res_dict = get_place_recognition_answer_logits(row, recog_prompt, tokenizer, 
                                                   text_key = "tweetFulltext",
                                                    engine = engine, 
                                                    n_tokens = n_tokens, 
                                                    temp=temp, 
                                                    logprobs = logprobs)
    
    with open(res_file, 'a') as f:
        f.write(json.dumps(res_dict) + "\n")
        
    # print the result for a preview
#     print("Tweet ID is: "+ res_dict['tweetFileID'])
#     print("Tweet text is: "+ res_dict['tweetFulltext'])
#     print("Annotation is: "+ str(res_dict['tweetAnnotation']))
#     print("Prediction is: "+ res_dict['answer'])
    
    time.sleep(0.5)
    
    # Test only retrieve 10
#     count +=1
#     if count == 11:
#         break

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:07:49<00:00,  4.07s/it]


### Use GPT-3 without geo-knowledge

In [19]:
recog_prompt = """
This is a set of location description recognition problems. 
The `Sentence` is a sentence containing location descriptions. The goal is to infer which parts of the sentence represent location descriptions and put them in double quotes.
--

--
Sentence: {TEXT}
Q: Which parts of this sentence represent location descriptions?
A:"""

In [20]:
data_dir = "data/"
anno_file = os.path.join(data_dir, 'tweetTextAndAnnotation.txt')
data_list = load_HarveyTweet2017_data(anno_file, data_dir, sep = "||")

In [23]:
engine = "text-davinci-002"  

set_seed(42)

n_tokens = 30
temp=0.2
logprobs = 5
logprobs_topk = 20

res_dir = "result/"  
if not os.path.exists(res_dir):
    os.mkdir(res_dir)
res_file = "{res_dir}/placerecognize_{engine}_{temp:.3f}_{n_tokens:d}_{logprobs:d}_no_geo.json".format(
        res_dir = res_dir,
        engine = engine,
        temp = temp,
        n_tokens = n_tokens,
        logprobs = logprobs
    )
res_dict_list = read_res_file(res_file)
process_ids = [res_dict['ID'] for res_dict in res_dict_list]

cur_data_list = [item for item in data_list if item["ID"] not in process_ids]
len(cur_data_list)

988

In [24]:
count = 0
for i, row in (pbar := tqdm(enumerate(cur_data_list), total= len(cur_data_list) )):
    res_dict = get_place_recognition_answer_logits(row, recog_prompt, tokenizer, 
                                                   text_key = "tweetFulltext",
                                                    engine = engine, 
                                                    n_tokens = n_tokens, 
                                                    temp=temp, 
                                                    logprobs = logprobs)
    
    original_answer = res_dict['answer']
    extracted_answer = re.findall('"([^"]*)"', original_answer)
    
    res_dict['origin_answer'] = res_dict['answer'] 
    res_dict['answer'] = ";".join(["CX: "+str(this_str) for this_str in extracted_answer])
    
    
    with open(res_file, 'a') as f:
        f.write(json.dumps(res_dict) + "\n")
        
    # print the result for a preview
#     print("Tweet ID is: "+ res_dict['tweetFileID'])
#     print("Tweet text is: "+ res_dict['tweetFulltext'])
#     print("Annotation is: "+ str(res_dict['tweetAnnotation']))
#     print("Original answer is: "+ res_dict['origin_answer'])
#     print("Answer is: "+ res_dict['answer'])
    
    count += 1
    time.sleep(0.5)
    
#     if count == 2:
#         break

f.close()

100%|██████████████████████████████████████████████████████████████████████████████| 988/988 [1:14:26<00:00,  4.52s/it]
